ATM: in: x out: 2

Wip1: in: x out: 2

Chk2: in: xy out: 1

Mdm2: in: x out: 1

p53: in: (x+y)z out: 2

1. Generate the stable oscillatory parameter regions

2. Analyze the projection of those parameter regions on each factor 

3. Generate the matching for each factor

4. Apply the factor matchings iteratively to generate set $A$

5. generate the $\Delta$ over $A$

6. Compute homology of $C_{*}(A)$

# Generate the stable oscillatory parameter regions

In [2]:
import sys
sys.path+=['/Users/lunzhang/anaconda3/lib/python3.6/site-packages']
import DSGRN
import os
import json

In [128]:
nwStr = "Mdm2:p53 \n ATM:~Wip1 \n Wip1:p53 \n Chk2:(ATM)(~Wip1) \n p53:(ATM+Chk2)(~Mdm2)"
network = DSGRN.Network(nwStr)
DSGRN.DrawGraph(network)
parameterGraph = DSGRN.ParameterGraph(network)
parameterGraph.size()

803520

In [129]:
parameterSample = parameterGraph.parameter(0)
domainGraph = DSGRN.DomainGraph(parameterSample)
morseDecomposition = DSGRN.MorseDecomposition(domainGraph.digraph())
morseGraph = DSGRN.MorseGraph(domainGraph, morseDecomposition)

In [130]:
oscillatory = []
size = parameterGraph.size()
for i in range(size):
    parameterSample = parameterGraph.parameter(i)
    domainGraph = DSGRN.DomainGraph(parameterSample)
    morseDecomposition = DSGRN.MorseDecomposition(domainGraph.digraph())
    morseGraph = DSGRN.MorseGraph(domainGraph, morseDecomposition)
    l = morseGraph.poset().size()
    if i%100000==0:
        print(i)
    for j in range(l):
        if morseGraph.annotation(j)[0].startswith('FC') and len(morseGraph.poset().children(j))==0:
            oscillatory.append(parameterSample)
            break

0
100000
200000


KeyboardInterrupt: 

In [ ]:
len(oscillatory)

In [135]:
# suppose there are at most 2 thresholds
def stringifyParser(l):
    ret =  []
    l = eval(l)
    #print(l)
    for i in range(len(l)):
        inputNo = l[i][1][0]  
        m=outputNo = l[i][1][1] 
        polyNo = pow(2,inputNo)
        totalLen = polyNo*outputNo

        logic = l[i][1][2]
        logic = bin(int(logic,16))[2:].zfill(totalLen)

        lower = []
        middle = []
        upper = []
        thetas = l[i][2]
        temp = []
        for j in range(0,len(logic),m):
            polyIndex = polyNo - int(j/m) - 1
            piece = logic[j:j+m]
            #print(piece)
            if len(set(piece)) == 2:
                middle.append(polyIndex)
            elif set(piece) == set('1'):
                upper.append(polyIndex)
            else:
                lower.append(polyIndex)

        if len(thetas)==1:
            temp = sorted(lower) + [-thetas[0]-1] + sorted(upper)
        else:
            
            temp = sorted(lower) + [-thetas[0]-1] + sorted(middle) + [-thetas[1]-1]+sorted(upper)
            #print(temp)

        ret.append(tuple(temp))
    return ret

In [ ]:
pnlist = [stringifyParser(x.stringify()) for x in oscillatory]

In [ ]:
pnlist[0],pnlist[1]

In [343]:
len(pnlist)

6904

# Analyze the projection of those parameter regions on each factor 

In [18]:
factorList = [set() for _ in range(5)]
for i in range(len(factorList)):
    for j in range(len(pnlist)):
        currPN = pnlist[j]
        factorList[i].add(currPN[i])

In [19]:
[len(x) for x in factorList]

[1, 6, 4, 4, 172]

In [258]:
factorList[1]

{(-2, 0, -1, 1),
 (-1, 0, -2, 1),
 (0, -2, -1, 1),
 (0, -2, 1, -1),
 (0, -1, -2, 1),
 (0, -1, 1, -2)}

In [16]:
# load the top cells from each atomic parameter node file
factorFiles = [x for x in os.listdir() if "_o_" in x and "PN" in x]

In [18]:
factorFiles

['xy_o_1_PN.json', 'x_o_2_PN.json', '(x+y)z_o_2_PN.json', 'x_o_1_PN.json']

In [19]:
factorFiles = ['x_o_1_PN.json','x_o_2_PN.json','x_o_2_PN.json','xy_o_1_PN.json','(x+y)z_o_2_PN.json']

In [20]:
atomicPNList = []
for i in range(len(factorFiles)):
    with open(factorFiles[i],'r') as file:
        tempPNList = json.load(file)
        currPNList = []
        for key in tempPNList:
            currPNList.append(eval(key))
        atomicPNList.append(currPNList)

In [21]:
[len(x) for x in atomicPNList]

[3, 12, 12, 6, 310]

In [279]:
factorIndex = 2
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = 0


for i in range(len(pnlist)):
    currPN = pnlist[i]
    currFactor = currPN[factorIndex]
    factorStatistics[currFactor]+=1

In [280]:
factorStatistics

{(0, -1, -2, 1): 2508,
 (0, -2, -1, 1): 2508,
 (-2, 0, -1, 1): 944,
 (0, -1, 1, -2): 944}

In [285]:
factorIndex = 2
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = set()

for i in range(len(pnlist)):
    currPN = pnlist[i]
    currFactor = currPN[factorIndex]
    rest = currPN[:factorIndex] + currPN[factorIndex+1:]
    rest = tuple(rest)
    factorStatistics[currFactor].add(rest)

In [288]:
s1 = factorStatistics[(0, -2, -1, 1)]
s2 = factorStatistics[(-2, 0, -1, 1)]
len(s1),len(s2)

(2508, 944)

In [290]:
s1 = factorStatistics[(0, -1, -2, 1)]
s2 = factorStatistics[(0, -1, 1, -2)]
len(s1),len(s2)

(2508, 944)

In [293]:
newPNList = []
s = set([(0, -1, -2, 1),(0, -2, -1, 1)])
for i in range(len(pnlist)):
    pn = pnlist[i]
    if pn[2] in s:
        newPNList.append(pn)

In [294]:
len(newPNList)

5016

In [344]:
# projection on the factor at 3
factorIndex = 3
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = 0

for i in range(len(newPNList)):
    currPN = newPNList[i]
    currFactor = currPN[factorIndex]
    factorStatistics[currFactor]+=1

In [345]:
factorStatistics

{(0, 1, 2, -1, 3): 1254,
 (0, 1, -1, 2, 3): 1564,
 (0, 2, -1, 1, 3): 944,
 (0, -1, 1, 2, 3): 1254}

In [346]:
factorIndex = 3
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = set()

for i in range(len(newPNList)):
    currPN = newPNList[i]
    currFactor = currPN[factorIndex]
    rest = currPN[:factorIndex] + currPN[factorIndex+1:]
    rest = tuple(rest)
    factorStatistics[currFactor].add(rest)
    


In [347]:
s1 = factorStatistics[(0, 1, 2, -1, 3)] 
s2 = factorStatistics[(0, 1, -1, 2, 3)]
s3 = factorStatistics[(0, 2, -1, 1, 3)]
s4 = factorStatistics[(0, -1, 1, 2, 3)]

In [353]:
print(s1 - s2, s4 - s2, s3 - s1)

set() set() set()


{((0, -1, 1),
  (-1, 0, -2, 1),
  (0, -2, -1, 1),
  (0, -2, -1, 1, 2, 3, 4, 5, 6, 7)),
 ((0, -1, 1),
  (-1, 0, -2, 1),
  (0, -2, -1, 1),
  (0, -2, 2, -1, 1, 3, 4, 5, 6, 7)),
 ((0, -1, 1),
  (-1, 0, -2, 1),
  (0, -2, -1, 1),
  (0, -2, 2, 4, -1, 1, 3, 5, 6, 7)),
 ((0, -1, 1),
  (-1, 0, -2, 1),
  (0, -2, -1, 1),
  (0, -2, 2, 4, 6, -1, 1, 3, 5, 7)),
 ((0, -1, 1),
  (-1, 0, -2, 1),
  (0, -2, -1, 1),
  (0, -2, 4, -1, 1, 2, 3, 5, 6, 7)),
 ((0, -1, 1),
  (-1, 0, -2, 1),
  (0, -2, -1, 1),
  (0, -1, -2, 1, 2, 3, 4, 5, 6, 7)),
 ((0, -1, 1),
  (-1, 0, -2, 1),
  (0, -2, -1, 1),
  (0, -1, 1, -2, 2, 3, 4, 5, 6, 7)),
 ((0, -1, 1),
  (-1, 0, -2, 1),
  (0, -2, -1, 1),
  (0, -1, 1, 2, -2, 3, 4, 5, 6, 7)),
 ((0, -1, 1),
  (-1, 0, -2, 1),
  (0, -2, -1, 1),
  (0, -1, 1, 2, 3, -2, 4, 5, 6, 7)),
 ((0, -1, 1),
  (-1, 0, -2, 1),
  (0, -2, -1, 1),
  (0, -1, 1, 2, 3, 4, -2, 5, 6, 7)),
 ((0, -1, 1),
  (-1, 0, -2, 1),
  (0, -2, -1, 1),
  (0, -1, 1, 2, 3, 4, 5, -2, 6, 7)),
 ((0, -1, 1),
  (-1, 0, -2, 1),
  (0, -2, -

In [320]:
# factor 1 
newNewPNList = []
s = set([(0, 1, -1, 2, 3)])
for i in range(len(newPNList)):
    pn = newPNList[i]
    if pn[3] in s:
        newNewPNList.append(pn)

In [321]:
len(newNewPNList)

1564

In [324]:
factorIndex = 1
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = 0

for i in range(len(newNewPNList)):
    currPN = newNewPNList[i]
    currFactor = currPN[factorIndex]
    factorStatistics[currFactor]+=1

In [325]:
factorStatistics

{(0, -1, -2, 1): 310,
 (0, -2, -1, 1): 310,
 (0, -1, 1, -2): 166,
 (0, -2, 1, -1): 310,
 (-1, 0, -2, 1): 310,
 (-2, 0, -1, 1): 158}

In [354]:
factorIndex = 1
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = set()

for i in range(len(newNewPNList)):
    currPN = newNewPNList[i]
    currFactor = currPN[factorIndex]
    rest = currPN[:factorIndex] + currPN[factorIndex+1:]
    rest = tuple(rest)
    factorStatistics[currFactor].add(rest)

In [356]:
s1 = factorStatistics[(0, -1, -2, 1)]
s2 = factorStatistics[(0, -2, -1, 1)]
s3 = factorStatistics[(0, -1, 1, -2)]
s4 = factorStatistics[(0, -2, 1, -1)]
s5 = factorStatistics[(-1, 0, -2, 1)]
s6 = factorStatistics[(-2, 0, -1, 1)]

In [359]:
s4-s2, s5 - s1

(set(), set())

In [360]:
s3 - s1

{((0, -1, 1),
  (0, -2, -1, 1),
  (0, 1, -1, 2, 3),
  (0, 1, 2, 3, 4, -1, 6, -2, 5, 7)),
 ((0, -1, 1),
  (0, -2, -1, 1),
  (0, 1, -1, 2, 3),
  (0, 1, 2, 4, -1, 3, 6, -2, 5, 7)),
 ((0, -1, 1),
  (0, -2, -1, 1),
  (0, 1, -1, 2, 3),
  (0, 1, 2, 4, -1, 6, -2, 3, 5, 7)),
 ((0, -1, 1),
  (0, -2, -1, 1),
  (0, 1, -1, 2, 3),
  (0, 1, 4, -1, 2, 6, -2, 3, 5, 7)),
 ((0, -1, 1),
  (0, -2, -1, 1),
  (0, 1, -1, 2, 3),
  (0, 2, 4, -1, 1, 3, 6, -2, 5, 7)),
 ((0, -1, 1),
  (0, -2, -1, 1),
  (0, 1, -1, 2, 3),
  (0, 2, 4, -1, 1, 6, -2, 3, 5, 7)),
 ((0, -1, 1),
  (0, -2, -1, 1),
  (0, 1, -1, 2, 3),
  (0, 2, 4, -1, 6, -2, 1, 3, 5, 7)),
 ((0, -1, 1),
  (0, -2, -1, 1),
  (0, 1, -1, 2, 3),
  (0, 4, -1, 1, 2, 6, -2, 3, 5, 7)),
 ((0, -1, 1),
  (0, -2, -1, 1),
  (0, 1, -1, 2, 3),
  (0, 4, -1, 2, 6, -2, 1, 3, 5, 7)),
 ((0, -1, 1),
  (0, -1, -2, 1),
  (0, 1, -1, 2, 3),
  (0, 1, 2, 3, 4, -1, 6, -2, 5, 7)),
 ((0, -1, 1),
  (0, -1, -2, 1),
  (0, 1, -1, 2, 3),
  (0, 1, 2, 4, -1, 3, 6, -2, 5, 7)),
 ((0, -1, 1),
  (0, -

In [335]:
len(s3), len(s3-s1), len(s3-s1-s2)

(166, 18, 18)

In [341]:
len(s6), len(s6-s1), len(s6-s1-s2)

(158, 16, 16)

In [272]:
factorIndex = 0
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = 0


for i in range(len(pnlist)):
    currPN = pnlist[i]
    currFactor = currPN[factorIndex]
    factorStatistics[currFactor]+=1

In [271]:
factorStatistics

{(0, 1, 4, 5, -2, 2, 6, -1, 3, 7): 32,
 (0, 1, 2, 4, -1, 5, -2, 3, 6, 7): 44,
 (0, 1, 2, 4, -2, 5, -1, 3, 6, 7): 44,
 (0, 4, -1, 1, 2, 5, -2, 3, 6, 7): 32,
 (0, -1, 1, 2, 3, -2, 4, 5, 6, 7): 44,
 (0, 1, 4, -1, 2, 3, 5, -2, 6, 7): 32,
 (0, 1, 4, -2, 2, 3, 5, 6, -1, 7): 32,
 (0, 1, 4, -2, 2, 3, 5, -1, 6, 7): 32,
 (0, -1, 1, 4, -2, 2, 3, 5, 6, 7): 32,
 (0, 1, -1, 2, 4, 5, -2, 3, 6, 7): 32,
 (0, 1, -2, 2, 4, 5, -1, 3, 6, 7): 32,
 (0, 1, 2, 4, -1, 6, -2, 3, 5, 7): 12,
 (0, 1, 2, 4, -2, 6, -1, 3, 5, 7): 56,
 (0, 4, -1, 1, 2, 6, -2, 3, 5, 7): 12,
 (0, 1, -1, 2, 3, 4, 5, -2, 6, 7): 32,
 (0, 1, -2, 2, 3, 4, 5, 6, -1, 7): 32,
 (0, 1, -2, 2, 3, 4, 5, -1, 6, 7): 32,
 (0, 1, 2, -1, 3, 4, 5, -2, 6, 7): 44,
 (0, 1, 2, -2, 3, 4, 5, 6, -1, 7): 32,
 (0, 1, 4, -1, 2, -2, 3, 5, 6, 7): 44,
 (0, 1, 2, 3, -1, 4, -2, 5, 6, 7): 44,
 (0, 1, 2, -2, 3, 4, 5, -1, 6, 7): 32,
 (0, 1, 4, -2, 2, -1, 3, 5, 6, 7): 44,
 (0, -1, 1, 2, -2, 3, 4, 5, 6, 7): 44,
 (0, 1, 2, 3, -2, 4, -1, 5, 6, 7): 56,
 (0, 1, 4, -2, 2, 5, 6, -

In [261]:
1860*2+620*2+996+948

6904

One observation is the first factor can be ignored as it is a cylindrical set over the only one atomic parameter node.

# Generate the matching for each factor

In [629]:
# Generate the facelattice for each atomic parameter node
factorFiles = [x for x in os.listdir() if "_o_" in x and "PN" in x]
atomicPNFaceLList = []
for i in range(len(factorFiles)):
    with open(factorFiles[i],'r') as file:
        tempPNList = json.load(file)
        atomicPNFaceLList.append(tempPNList)

In [632]:
factorFiles

['xy_o_1_PN.json', 'x_o_2_PN.json', '(x+y)z_o_2_PN.json', 'x_o_1_PN.json']

In [631]:
atomicPNFaceLList[0]

{'(-1, 0, 1, 2, 3)': {'0': [[9], [10], [5], [17], [12], [7]],
  '1': [[5, 9],
   [10, 17],
   [7, 12],
   [5, 17],
   [5, 12],
   [5, 7],
   [10, 12],
   [9, 10],
   [9, 17],
   [7, 10],
   [7, 17],
   [9, 12],
   [7, 9]],
  '2': [[9, 10, 17],
   [5, 7, 9],
   [9, 10, 12],
   [7, 9, 17],
   [7, 9, 10],
   [7, 10, 17],
   [5, 7, 17],
   [5, 9, 17],
   [5, 7, 12],
   [7, 10, 12],
   [5, 9, 12],
   [7, 9, 12],
   [5, 10, 12, 17]],
  '3': [[7, 9, 10, 12],
   [7, 9, 10, 17],
   [5, 7, 10, 12, 17],
   [5, 9, 10, 12, 17],
   [5, 7, 9, 17],
   [5, 7, 9, 12]],
  '4': [[5, 7, 9, 10, 12, 17]]},
 '(0, -1, 1, 2, 3)': {'0': [[0], [3], [9], [5], [16], [11], [6], [12], [7]],
  '1': [[11, 16],
   [5, 9],
   [6, 9],
   [3, 12],
   [5, 16],
   [6, 11],
   [7, 12],
   [5, 12],
   [5, 7],
   [3, 11],
   [0, 7],
   [0, 11],
   [0, 16],
   [3, 6],
   [3, 16],
   [0, 3],
   [6, 16],
   [9, 12],
   [7, 9]],
  '2': [[0, 11, 16],
   [3, 11, 16],
   [6, 11, 16],
   [3, 6, 9, 12],
   [0, 5, 7, 16],
   [0, 6, 7, 9,

In [4]:
def isFace(f,c):
    if set(f).issubset(set(c)):
        return True
    
    return False

def getFaceLattice(apnList):
    faceLattice = dict()
    for index in apnList:
        pn = apnList[index]
        d = len(pn)
        for dim in range(d-1,0,-1):
            cells = pn[str(dim)]
            faces = pn[str(dim-1)]
            for c in cells:
                for f in faces:
                    if isFace(f,c):
                        sf = tuple(sorted(f))
                        sc = str(tuple(sorted(c)))
                        if sc not in faceLattice:
                            faceLattice[sc] = []
                        faceLattice[sc].append(sf)
        
        # the zero diminsion
        cells = pn['0']
        for point in cells:
            faceLattice[str(tuple(sorted(point)))] = []
            
            
    # remove the duplicate faces
    for key in faceLattice:
        faceLattice[key] = list(set(faceLattice[key]))
            
    return faceLattice

In [634]:
factorFiles

['xy_o_1_PN.json', 'x_o_2_PN.json', '(x+y)z_o_2_PN.json', 'x_o_1_PN.json']

In [5]:
# save the facelattice
factorFiles = [x for x in os.listdir() if "_o_" in x and "PN" in x]

atomicPNFaceLList = []
for i in range(len(factorFiles)):
    with open(factorFiles[i],'r') as file:
        tempPNList = json.load(file)
        atomicPNFaceLList.append(tempPNList)

for i in range(len(factorFiles)):
    print(i)
    filename = factorFiles[i].split('_PN')[0] + '_FL.dat'
    res = getFaceLattice(atomicPNFaceLList[i])
    with open(filename,'w') as file:
        json.dump(res,file)  

0
1
2
3


In [6]:
atomicPNFaceLList[-1]

{'(-1, 0, 1)': {'0': [[3], [6], [1], [4]],
  '1': [[3, 4], [1, 3], [1, 6], [4, 6]],
  '2': [[1, 3, 4, 6]]},
 '(0, 1, -1)': {'0': [[2], [5], [0]],
  '1': [[2, 5], [0, 5], [0, 2]],
  '2': [[0, 2, 5]]},
 '(0, -1, 1)': {'0': [[2], [0], [3], [1]],
  '1': [[1, 2], [0, 3], [1, 3], [0, 2]],
  '2': [[0, 1, 2, 3]]}}

In [637]:
# load the face lattice
latticeFiles = [x for x in os.listdir() if "FL" in x]
print(latticeFiles)
apnFaceLatticeList = []
for i in range(len(latticeFiles)):
    fileName = latticeFiles[i]
    with open(fileName,'r') as file:
        temp = json.load(file)
        apnFaceLatticeList.append(temp)

['xy_o_1_FL.dat', 'x_o_2_FL.dat', '(x+y)z_o_2_FL.dat', 'x_o_1_FL.dat']


In [7]:
# load the face lattice/ boundary matrix
latticeFiles = [x for x in os.listdir() if "FL" in x]
print(latticeFiles)
apnFaceLatticeList = []
for i in range(len(latticeFiles)):
    fileName = latticeFiles[i]
    with open(fileName,'r') as file:
        temp = json.load(file)
        apnFaceLatticeList.append(temp)

        
for i in range(len(apnFaceLatticeList)):
    apnFaceLattice = apnFaceLatticeList[i]
    temp = dict()
    for key in apnFaceLattice:
        tempVal = apnFaceLattice[key]
        tempVal = [tuple(x) for x in tempVal]
        temp[eval(key)] = set(tempVal)
    apnFaceLatticeList[i] = temp

['xy_o_1_FL.dat', 'x_o_2_FL.dat', '(x+y)z_o_2_FL.dat', 'x_o_1_FL.dat']


In [8]:
# generate coboundary matrix 
coboundaryMatrixList = []
for i in range(len(apnFaceLatticeList)):
    boundaryMatrix = apnFaceLatticeList[i]
    coboundaryMatrix = dict()
    for cell in boundaryMatrix:
        boundary = boundaryMatrix[cell]
        for b in boundary:
            if str(b) not in coboundaryMatrix:
                coboundaryMatrix[str(b)] = []
            coboundaryMatrix[str(b)].append(cell)
    coboundaryMatrixList.append(coboundaryMatrix)

for i in range(len(latticeFiles)):
    filename = latticeFiles[i].split("_FL")[0] + "_CB.dat"
    with open(filename, 'w') as file:
        json.dump(coboundaryMatrixList[i],file)

In [640]:
# load coboundary matrix
coboundaryFiles = [x for x in os.listdir() if "CB" in x]
print(coboundaryFiles)
coboundaryMatrixList = []
for i in range(len(coboundaryFiles)):
    fileName = coboundaryFiles[i]
    with open(fileName,'r') as file:
        temp = json.load(file)
        coboundaryMatrixList.append(temp)

        
for i in range(len(coboundaryMatrixList)):
    apnCoboundary = coboundaryMatrixList[i]
    temp = dict()
    for key in apnCoboundary:
        tempVal = apnCoboundary[key]
        tempVal = [tuple(x) for x in tempVal]
        temp[eval(key)] = set(tempVal)
    coboundaryMatrixList[i] = temp

['(x+y)z_o_2_CB.dat', 'x_o_2_CB.dat', 'xy_o_1_CB.dat', 'x_o_1_CB.dat']


In [641]:
def checkCoreduction(face, coboundary, currComplex,used):
    for cell in coboundary:        
        if len(currComplex[cell] - used) == 1:
            return True
    return False


def genMatching(currComplex,coboundaryMatrix):
    
    used = set()
    kings = []
    queens = []
    aces = []
    
    count = 0
    while currComplex:
        count +=1
        
        # find queen, king pairs
        flag = True
        
        # flag: find coreducible pair in previous around
        while flag:
            tempComplex = dict()
            flag = False
            for cell in currComplex:

                tempDiff = currComplex[cell] - used

                if len(tempDiff) == 1:
                    queen = list(tempDiff)[0]
                    king = cell

                    queens.append(queen)
                    kings.append(king)

                    used.add(king)
                    used.add(queen)

                    flag = True

                if cell not in used:
                    tempComplex[cell] = tempDiff
            
            currComplex = tempComplex
            
        #print(queens, kings, aces)
        #print(tempComplex)
        
        # check coreducible or not
        flag = False # exist coreducible or not
        while not flag:
            tempComplex = dict()
            
            for key in currComplex:
                
                # if already have coreduction in previous loop, then only restore the rest of dictionary
                if flag:
                    tempComplex[key] = currComplex[key]
                    continue
                
                
                if len(currComplex[key]) == 0:
                    aces.append(key)
                    used.add(key)

                    # if key is a top cell, just continue
                    if key not in coboundaryMatrix:
                        continue

                    # if there is a coreducible pair we break the aces collection process
                    if checkCoreduction(key, coboundaryMatrix[key], currComplex, used):
                        flag = True
                        
                else:
                    tempComplex[key] = currComplex[key]
            
            currComplex = tempComplex
            
            # if empty
            if not currComplex:
                break
        
    return queens, kings, aces

In [105]:
print(latticeFiles)
print(coboundaryFiles)

['xy_o_1_FL.dat', 'x_o_2_FL.dat', '(x+y)z_o_2_FL.dat', 'x_o_1_FL.dat']
['(x+y)z_o_2_CB.dat', 'x_o_2_CB.dat', 'xy_o_1_CB.dat', 'x_o_1_CB.dat']


In [642]:
# generate all the matchings (K,Q,A)
for i in range(len(factorFiles)):
    pn = factorFiles[i].split('_PN')[0]
    flIndex = 0
    for i in range(len(latticeFiles)):
        if pn in latticeFiles[i]:
            flIndex = i
            break
    cbIndex = 0
    for i in range(len(coboundaryFiles)):
        if pn in coboundaryFiles[i]:
            cbIndex = i
            break
    #print(flIndex, cbIndex)  
    
    currComplex = apnFaceLatticeList[flIndex]
    coboundaryMatrix = coboundaryMatrixList[cbIndex]
    matchings = genMatching(currComplex,coboundaryMatrix)
    
    fileName = pn+'_MA.dat'
    
    with open(fileName,'w') as file:
        json.dump(matchings, file)

In [643]:
print(latticeFiles)
print(coboundaryFiles)

['xy_o_1_FL.dat', 'x_o_2_FL.dat', '(x+y)z_o_2_FL.dat', 'x_o_1_FL.dat']
['(x+y)z_o_2_CB.dat', 'x_o_2_CB.dat', 'xy_o_1_CB.dat', 'x_o_1_CB.dat']


# Apply the factor matchings iteratively to generate set $A$

In [644]:
oscillatory = []
with open('osciParameter.dat','r') as file:
    line = file.readline()
    while line:
        oscillatory.append(line)
        line = file.readline()

pnlist = [stringifyParser(x) for x in oscillatory]
pnlist[0]

In [645]:
oscillatory[0]

'[["Mdm2",[1,1,"2"],[0]],["ATM",[1,2,"C"],[0,1]],["Wip1",[1,2,"4"],[0,1]],["Chk2",[2,1,"8"],[0]],["p53",[3,2,"C000"],[0,1]]]\n'

In [646]:
pnlist = [stringifyParser(x) for x in oscillatory]
pnlist[0]

[(0, -1, 1),
 (0, -1, -2, 1),
 (0, -1, 1, -2),
 (0, 1, 2, -1, 3),
 (0, 1, 2, 3, 4, 5, 6, -1, -2, 7)]

In [647]:
factorList = [set() for _ in range(5)]
for i in range(len(factorList)):
    for j in range(len(pnlist)):
        currPN = pnlist[j]
        factorList[i].add(currPN[i])

In [648]:
print([len(x) for x in factorList])
print([len(x) for x in atomicPNList])

[1, 6, 4, 4, 172]
[3, 12, 12, 6, 310]


In [649]:
# we can remove the Mdm2 x 1 out node, i.e the first component
reducePNList = [x[1:] for x in pnlist]

In [1]:
def loadAtomicChainComplex(fpath):
    ret = dict()
    with open(fpath,'r') as file:
        atomicCC = json.load(file)
    for key in atomicCC:
        newk = key[1:-1].split(',')
        newk = tuple([int(x) for x in newk])
        v = atomicCC[key]
        newvalue = dict()
        for dim in v:
            newfaces = set([tuple(x) for x in v[dim]])
            newvalue[int(dim)] = newfaces
        ret[newk] = newvalue 
    return ret

In [651]:
factorFiles = ['x_o_2_PN.json','x_o_2_PN.json','xy_o_1_PN.json','(x+y)z_o_2_PN.json']

In [652]:
apnComplex = []
for i in range(len(factorFiles)):
    fpath = factorFiles[i]
    apnComplex.append(loadAtomicChainComplex(fpath))

In [653]:
matchingFactor = 'x_o_2_MA.dat'
with open(matchingFactor,'r') as file:
    matchings = json.load(file)

In [654]:
queens, kings, aces = matchings

In [203]:
pairQK = dict()
for i in range(len(queens)):
    pairQK[tuple(queens[i])] = tuple(kings[i])

In [219]:
factorIndex = 0
key = reducePNList[0][factorIndex]
currFactorComplex = apnComplex[factorIndex][key]

In [206]:
temp = set()
for key in currFactorComplex:
    temp|= currFactorComplex[key]
currFactorComplex = temp

In [208]:
used = set()
for cell in currFactorComplex:
    
    if cell in used:
        continue
    
    if cell in pairQK and pairQK[cell] in currFactorComplex:
        #print(cell, pairQK[cell])
        used.add(cell)
        used.add(pairQK[cell])
aces = currFactorComplex - used

In [242]:
matchingFactor = 'x_o_2_MA.dat'
with open(matchingFactor,'r') as file:
    matchings = json.load(file)

queens, kings, aces = matchings
pairQK = dict()
for i in range(len(queens)):
    pairQK[tuple(queens[i])] = tuple(kings[i])



for i in range(len(reducePNList)):
 
    factorIndex = 0
    key = reducePNList[i][factorIndex]
    currFactorComplex = apnComplex[factorIndex][key]
    
    temp = set()
    for key in currFactorComplex:
        temp|= currFactorComplex[key]
    currFactorComplex = temp
    
    used = set()
    for cell in currFactorComplex:

        if cell in used:
            continue

        if cell in pairQK and pairQK[cell] in currFactorComplex:
            #print(cell, pairQK[cell])
            used.add(cell)
            used.add(pairQK[cell])
    aces = currFactorComplex - used
    
    print(i, len(aces),len(currFactorComplex))

0 1 21
1 1 21
2 3 27
3 1 21
4 1 21
5 1 21
6 3 27
7 1 21
8 1 21
9 1 21
10 3 27
11 1 21
12 1 21
13 1 21
14 3 27
15 1 21
16 1 21
17 1 21
18 3 27
19 1 21
20 1 21
21 1 21
22 3 27
23 1 21
24 1 21
25 1 21
26 3 27
27 1 21
28 1 21
29 1 21
30 3 27
31 1 21
32 1 21
33 1 21
34 3 27
35 1 21
36 1 21
37 1 21
38 3 27
39 1 21
40 1 21
41 1 21
42 3 27
43 1 21
44 1 21
45 1 21
46 3 27
47 1 21
48 1 21
49 1 21
50 3 27
51 1 21
52 1 21
53 1 21
54 3 27
55 1 21
56 1 21
57 1 21
58 3 27
59 1 21
60 1 21
61 1 21
62 3 27
63 1 21
64 5 21
65 1 21
66 5 21
67 1 21
68 3 27
69 5 21
70 1 21
71 5 21
72 1 21
73 5 21
74 1 21
75 3 27
76 5 21
77 1 21
78 5 21
79 1 21
80 5 21
81 1 21
82 3 27
83 5 21
84 1 21
85 5 21
86 1 21
87 5 21
88 1 21
89 3 27
90 5 21
91 1 21
92 5 21
93 1 21
94 5 21
95 1 21
96 3 27
97 5 21
98 1 21
99 5 21
100 1 21
101 5 21
102 1 21
103 3 27
104 5 21
105 1 21
106 5 21
107 1 21
108 5 21
109 1 21
110 3 27
111 5 21
112 1 21
113 5 21
114 1 21
115 5 21
116 1 21
117 3 27
118 5 21
119 1 21
120 5 21
121 1 21
122 5 21
123

1172 5 21
1173 3 21
1174 5 21
1175 3 21
1176 3 21
1177 3 21
1178 3 21
1179 3 21
1180 5 21
1181 3 21
1182 5 21
1183 3 21
1184 3 21
1185 5 27
1186 3 21
1187 5 27
1188 3 21
1189 5 27
1190 3 21
1191 5 27
1192 5 21
1193 3 21
1194 5 27
1195 5 21
1196 3 21
1197 5 27
1198 3 21
1199 5 27
1200 3 21
1201 5 27
1202 3 21
1203 5 27
1204 3 21
1205 5 27
1206 5 21
1207 3 21
1208 5 27
1209 5 21
1210 3 21
1211 5 27
1212 3 21
1213 3 21
1214 3 21
1215 3 21
1216 5 21
1217 3 21
1218 5 21
1219 3 21
1220 3 21
1221 3 21
1222 3 21
1223 3 21
1224 5 21
1225 3 21
1226 5 21
1227 3 21
1228 3 21
1229 3 21
1230 3 21
1231 3 21
1232 5 21
1233 3 21
1234 5 21
1235 3 21
1236 3 21
1237 3 21
1238 3 21
1239 3 21
1240 5 21
1241 3 21
1242 5 21
1243 3 21
1244 3 21
1245 3 21
1246 3 21
1247 3 21
1248 5 21
1249 3 21
1250 5 21
1251 3 21
1252 3 21
1253 3 21
1254 3 21
1255 3 21
1256 5 21
1257 3 21
1258 5 21
1259 3 21
1260 3 21
1261 5 27
1262 3 21
1263 5 27
1264 3 21
1265 5 27
1266 3 21
1267 5 27
1268 5 21
1269 3 21
1270 5 27
1271 5 21


2172 1 21
2173 3 27
2174 5 21
2175 1 21
2176 5 21
2177 1 21
2178 5 21
2179 1 21
2180 3 27
2181 5 21
2182 1 21
2183 5 21
2184 1 21
2185 5 21
2186 1 21
2187 3 27
2188 5 21
2189 1 21
2190 5 21
2191 1 21
2192 5 21
2193 1 21
2194 3 27
2195 5 21
2196 1 21
2197 5 21
2198 1 21
2199 5 21
2200 1 21
2201 3 27
2202 5 21
2203 1 21
2204 5 21
2205 1 21
2206 1 21
2207 3 27
2208 1 21
2209 1 21
2210 1 21
2211 3 27
2212 1 21
2213 1 21
2214 1 21
2215 3 27
2216 1 21
2217 1 21
2218 1 21
2219 3 27
2220 1 21
2221 1 21
2222 1 21
2223 3 27
2224 1 21
2225 1 21
2226 1 21
2227 3 27
2228 1 21
2229 1 21
2230 1 21
2231 3 27
2232 1 21
2233 1 21
2234 1 21
2235 3 27
2236 1 21
2237 1 21
2238 1 21
2239 3 27
2240 1 21
2241 1 21
2242 1 21
2243 3 27
2244 1 21
2245 1 21
2246 1 21
2247 3 27
2248 1 21
2249 1 21
2250 1 21
2251 3 27
2252 1 21
2253 1 21
2254 1 21
2255 3 27
2256 1 21
2257 1 21
2258 1 21
2259 3 27
2260 1 21
2261 1 21
2262 1 21
2263 3 27
2264 1 21
2265 1 21
2266 1 21
2267 3 27
2268 1 21
2269 1 21
2270 1 21
2271 3 27


3338 3 21
3339 3 21
3340 3 21
3341 3 21
3342 3 21
3343 5 21
3344 3 21
3345 5 21
3346 3 21
3347 3 21
3348 1 21
3349 1 21
3350 3 27
3351 1 21
3352 1 21
3353 1 21
3354 3 27
3355 1 21
3356 1 21
3357 1 21
3358 3 27
3359 1 21
3360 1 21
3361 1 21
3362 3 27
3363 1 21
3364 1 21
3365 1 21
3366 3 27
3367 1 21
3368 1 21
3369 1 21
3370 3 27
3371 1 21
3372 1 21
3373 1 21
3374 3 27
3375 1 21
3376 1 21
3377 1 21
3378 3 27
3379 1 21
3380 1 21
3381 1 21
3382 3 27
3383 1 21
3384 1 21
3385 1 21
3386 3 27
3387 1 21
3388 1 21
3389 1 21
3390 3 27
3391 1 21
3392 1 21
3393 1 21
3394 3 27
3395 1 21
3396 5 21
3397 1 21
3398 5 21
3399 1 21
3400 3 27
3401 5 21
3402 1 21
3403 5 21
3404 1 21
3405 5 21
3406 1 21
3407 3 27
3408 5 21
3409 1 21
3410 5 21
3411 1 21
3412 5 21
3413 1 21
3414 3 27
3415 5 21
3416 1 21
3417 5 21
3418 1 21
3419 5 21
3420 1 21
3421 3 27
3422 5 21
3423 1 21
3424 5 21
3425 1 21
3426 5 21
3427 1 21
3428 3 27
3429 5 21
3430 1 21
3431 5 21
3432 1 21
3433 5 21
3434 1 21
3435 3 27
3436 5 21
3437 1 21


4505 3 21
4506 3 21
4507 3 21
4508 5 21
4509 3 21
4510 5 21
4511 3 21
4512 3 21
4513 3 21
4514 3 21
4515 3 21
4516 5 21
4517 3 21
4518 5 21
4519 3 21
4520 3 21
4521 3 21
4522 3 21
4523 3 21
4524 5 21
4525 3 21
4526 5 21
4527 3 21
4528 3 21
4529 5 27
4530 3 21
4531 5 27
4532 3 21
4533 5 27
4534 3 21
4535 5 27
4536 5 21
4537 3 21
4538 5 27
4539 5 21
4540 3 21
4541 5 27
4542 3 21
4543 3 21
4544 3 21
4545 3 21
4546 5 21
4547 3 21
4548 5 21
4549 3 21
4550 3 21
4551 5 27
4552 3 21
4553 5 27
4554 3 21
4555 5 27
4556 3 21
4557 5 27
4558 5 21
4559 3 21
4560 5 27
4561 5 21
4562 3 21
4563 5 27
4564 5 27
4565 5 27
4566 5 27
4567 5 27
4568 5 27
4569 5 27
4570 3 21
4571 5 27
4572 3 21
4573 5 27
4574 3 21
4575 5 27
4576 3 21
4577 5 27
4578 5 21
4579 3 21
4580 5 27
4581 5 21
4582 3 21
4583 5 27
4584 3 21
4585 3 21
4586 3 21
4587 3 21
4588 5 21
4589 3 21
4590 5 21
4591 3 21
4592 3 21
4593 3 21
4594 3 21
4595 3 21
4596 5 21
4597 3 21
4598 5 21
4599 3 21
4600 3 21
4601 5 27
4602 3 21
4603 5 27
4604 3 21


5504 1 21
5505 3 27
5506 5 21
5507 1 21
5508 5 21
5509 1 21
5510 5 21
5511 1 21
5512 3 27
5513 5 21
5514 1 21
5515 5 21
5516 1 21
5517 5 21
5518 1 21
5519 3 27
5520 5 21
5521 1 21
5522 5 21
5523 1 21
5524 1 21
5525 3 27
5526 1 21
5527 1 21
5528 1 21
5529 3 27
5530 1 21
5531 1 21
5532 1 21
5533 3 27
5534 1 21
5535 1 21
5536 1 21
5537 3 27
5538 1 21
5539 1 21
5540 1 21
5541 3 27
5542 1 21
5543 1 21
5544 1 21
5545 3 27
5546 1 21
5547 1 21
5548 1 21
5549 3 27
5550 1 21
5551 1 21
5552 1 21
5553 3 27
5554 1 21
5555 1 21
5556 1 21
5557 3 27
5558 1 21
5559 1 21
5560 1 21
5561 3 27
5562 1 21
5563 1 21
5564 1 21
5565 3 27
5566 1 21
5567 1 21
5568 1 21
5569 3 27
5570 1 21
5571 1 21
5572 1 21
5573 3 27
5574 1 21
5575 1 21
5576 1 21
5577 3 27
5578 1 21
5579 1 21
5580 1 21
5581 3 27
5582 1 21
5583 1 21
5584 1 21
5585 3 27
5586 1 21
5587 1 21
5588 1 21
5589 3 27
5590 1 21
5591 1 21
5592 1 21
5593 3 27
5594 1 21
5595 1 21
5596 1 21
5597 3 27
5598 1 21
5599 1 21
5600 1 21
5601 3 27
5602 1 21
5603 1 21


6504 3 21
6505 5 21
6506 3 21
6507 5 21
6508 3 21
6509 3 21
6510 3 21
6511 3 21
6512 3 21
6513 5 21
6514 3 21
6515 5 21
6516 3 21
6517 3 21
6518 3 21
6519 5 27
6520 3 21
6521 5 27
6522 3 21
6523 5 27
6524 5 21
6525 3 21
6526 5 27
6527 5 21
6528 3 21
6529 5 27
6530 3 21
6531 5 27
6532 3 21
6533 3 21
6534 3 21
6535 5 21
6536 3 21
6537 5 21
6538 3 21
6539 3 21
6540 3 21
6541 3 21
6542 3 21
6543 5 21
6544 3 21
6545 5 21
6546 3 21
6547 3 21
6548 3 21
6549 5 27
6550 3 21
6551 5 27
6552 3 21
6553 5 27
6554 5 21
6555 3 21
6556 5 27
6557 5 21
6558 3 21
6559 5 27
6560 3 21
6561 5 27
6562 3 21
6563 3 21
6564 3 21
6565 5 21
6566 3 21
6567 5 21
6568 3 21
6569 3 21
6570 3 21
6571 3 21
6572 3 21
6573 5 21
6574 3 21
6575 5 21
6576 3 21
6577 3 21
6578 3 21
6579 5 27
6580 3 21
6581 5 27
6582 3 21
6583 5 27
6584 5 21
6585 3 21
6586 5 27
6587 5 21
6588 3 21
6589 5 27
6590 3 21
6591 5 27
6592 3 21
6593 3 21
6594 3 21
6595 5 21
6596 3 21
6597 5 21
6598 3 21
6599 3 21
6600 3 21
6601 3 21
6602 3 21
6603 5 21


In [227]:
factorIndex = 0
pnIndex = 182
key = reducePNList[pnIndex][factorIndex]
currFactorComplex = apnComplex[factorIndex][key]

temp = set()
for key in currFactorComplex:
    temp|= currFactorComplex[key]
currFactorComplex = temp

used = set()
for cell in currFactorComplex:

    if cell in used:
        continue

    if cell in pairQK and pairQK[cell] in currFactorComplex:
        #print(cell, pairQK[cell])
        used.add(cell)
        used.add(pairQK[cell])
aces = currFactorComplex - used

print(i, len(aces),len(currFactorComplex))

6903 5 21


In [228]:
aces

{(11,), (11, 12, 16, 18), (11, 16), (11, 18), (18,)}

In [233]:
def integerPartitionHelper(Sum,UpperBounds,path,ret):
    if len(path) == len(UpperBounds):
        if Sum!=0:
            return
        else:
            ret.append(path.copy())
            return 
    
    for i in range(min(Sum,UpperBounds[len(path)])+1):
        path.append(i)
        integerPartitionHelper(Sum-i,UpperBounds,path,ret)
        #print(path)
        path.pop()
        
    return 
            

def integerPartition(Sum, UpperBounds):
    ret = []
    path = []
    integerPartitionHelper(Sum,UpperBounds,path,ret)
    return ret

In [240]:
for i in range(17):
    print(len(integerPartition(i,[3,3,4,7])))

1
4
10
20
33
47
60
70
75
75
70
60
47
33
20
10
4


In [ ]:
# generate the aces for single global parameter node
def genAcesSinglePN(pn ,matchings):
    currComplex = pn
    
    

# Reduction method

The order of the node is Mdm2, ATM, Wip1, Chk2, p53

In [655]:
oscillatory = []
with open('osciParameter.dat','r') as file:
    line = file.readline()
    while line:
        oscillatory.append(line)
        line = file.readline()

In [656]:
pnlist = [stringifyParser(x) for x in oscillatory]
pnlist[0]

[(0, -1, 1),
 (0, -1, -2, 1),
 (0, -1, 1, -2),
 (0, 1, 2, -1, 3),
 (0, 1, 2, 3, 4, 5, 6, -1, -2, 7)]

In [657]:
factorList = [set() for _ in range(5)]
for i in range(len(factorList)):
    for j in range(len(pnlist)):
        currPN = pnlist[j]
        factorList[i].add(currPN[i])

In [658]:
print([len(x) for x in factorList])
print([len(x) for x in atomicPNList])

[1, 6, 4, 4, 172]
[3, 12, 12, 6, 310]


In [661]:
factorFiles[2]

'xy_o_1_PN.json'

## Mdm2

From above computation, the Mdm2 is can be removed by cylindrical set structure

In [659]:
# Mdm2 term removed
pnlist0r = [x[1:] for x in pnlist]

## Wip1 

In [662]:
factorIndex = 2
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = 0


for i in range(len(pnlist)):
    currPN = pnlist[i]
    currFactor = currPN[factorIndex]
    factorStatistics[currFactor]+=1

In [663]:
factorStatistics

{(0, -1, -2, 1): 2508,
 (0, -2, -1, 1): 2508,
 (-2, 0, -1, 1): 944,
 (0, -1, 1, -2): 944}

In [664]:
factorIndex = 2
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = set()

for i in range(len(pnlist)):
    currPN = pnlist[i]
    currFactor = currPN[factorIndex]
    rest = currPN[:factorIndex] + currPN[factorIndex+1:]
    rest = tuple(rest)
    factorStatistics[currFactor].add(rest)

In [665]:
s1 = factorStatistics[(0, -1, -2, 1)]
s2 = factorStatistics[(0, -2, -1, 1)]
s3 = factorStatistics[(-2, 0, -1, 1)]
s4 = factorStatistics[(0, -1, 1, -2)]

print(s4-s1, s3-s2, s1-s2)

set() set() set()


In [666]:
# show the polyhedra complex structure
print(s4-s1, s3-s2, s1-s2)

set() set() set()


In [667]:
def checkCoreduction(face, coboundary, currComplex,used):
    for cell in coboundary:        
        if len(currComplex[cell] - used) == 1:
            return True
    return False


# used is the preset attractors
def genMatchingWithPreAtt(currComplex,coboundaryMatrix,used):
    
    kings = []
    queens = []
    aces = []
    
    count = 0
    while currComplex:
        count +=1
        
        # find queen, king pairs
        flag = True
        
        # flag: find coreducible pair in previous around
        while flag:
            tempComplex = dict()
            flag = False
            for cell in currComplex:

                tempDiff = currComplex[cell] - used

                if len(tempDiff) == 1:
                    queen = list(tempDiff)[0]
                    king = cell

                    queens.append(queen)
                    kings.append(king)

                    used.add(king)
                    used.add(queen)

                    flag = True

                if cell not in used:
                    tempComplex[cell] = tempDiff
            
            currComplex = tempComplex
            
        #print(queens, kings, aces)
        #print(tempComplex)
        
        # check coreducible or not
        flag = False # exist coreducible or not
        while not flag:
            tempComplex = dict()
            
            for key in currComplex:
                
                # if already have coreduction in previous loop, then only restore the rest of dictionary
                if flag:
                    tempComplex[key] = currComplex[key]
                    continue
                
                
                if len(currComplex[key]) == 0:
                    aces.append(key)
                    used.add(key)

                    # if key is a top cell, just continue
                    if key not in coboundaryMatrix:
                        continue

                    # if there is a coreducible pair we break the aces collection process
                    if checkCoreduction(key, coboundaryMatrix[key], currComplex, used):
                        flag = True
                        
                else:
                    tempComplex[key] = currComplex[key]
            
            currComplex = tempComplex
            
            # if empty
            if not currComplex:
                break
        
    return queens, kings, aces

def isFace(f,c):
    if set(f).issubset(set(c)):
        return True
    
    return False

def getBoundaryMatrix(apnList):
    faceLattice = dict()
    for index in apnList:
        pn = apnList[index]
        d = len(pn)
        for dim in range(d-1,0,-1):
            cells = pn[dim]
            faces = pn[dim-1]
            for c in cells:
                for f in faces:
                    if isFace(f,c):
                        sf = tuple(sorted(f))
                        sc = tuple(sorted(c))
                        if sc not in faceLattice:
                            faceLattice[sc] = []
                        faceLattice[sc].append(sf)
        
        # the zero diminsion
        cells = pn[0]
        for point in cells:
            faceLattice[tuple(sorted(point))] = []
            
            
    # remove the duplicate faces
    for key in faceLattice:
        faceLattice[key] = list(set(faceLattice[key]))
            
    return faceLattice

def getCoboundaryMatrix(boundaryMatrix):
    coboundaryMatrix = dict()
    
    for cell in boundaryMatrix:
        boundary = boundaryMatrix[cell]
        for b in boundary:
            if b not in coboundaryMatrix:
                coboundaryMatrix[b] = []
            coboundaryMatrix[b].append(cell)
    return coboundaryMatrix

   

construct the matching over the Wip1 factor space, with preset attractors

In [ ]:
apnList = loadAtomicChainComplex('x_o_2_PN.json')
factorAPN = dict()
for k in factorStatistics:
    factorAPN[k] = apnList[k]
    
s1 = factorStatistics[(0, -1, -2, 1)]
s2 = factorStatistics[(0, -2, -1, 1)]
s3 = factorStatistics[(-2, 0, -1, 1)]
s4 = factorStatistics[(0, -1, 1, -2)]

boundaryMatrix = getBoundaryMatrix(factorAPN)
coboundaryMatrix = getCoboundaryMatrix(boundaryMatrix)

preAtt = set()
for i in [(0, -1, -2, 1), (0, -2, -1, 1)]:
    apn = factorAPN[i]
    for d in apn:
        preAtt|= apn[d]
        
tempComplex = set()
for i in factorAPN:
    apn = factorAPN[i]
    for d in apn:
        tempComplex|= apn[d]
tempComplex-=preAtt

subComplex = dict()
for cell in tempComplex:
    subComplex[cell] = set(boundaryMatrix[cell]) - preAtt
    
queens, kings, aces = genMatchingWithPreAtt(subComplex,coboundaryMatrix,preAtt)


In [668]:
apnList = loadAtomicChainComplex('x_o_2_PN.json')
factorAPN = dict()
for k in factorStatistics:
    factorAPN[k] = apnList[k]

In [669]:
s1 = factorStatistics[(0, -1, -2, 1)]
s2 = factorStatistics[(0, -2, -1, 1)]
s3 = factorStatistics[(-2, 0, -1, 1)]
s4 = factorStatistics[(0, -1, 1, -2)]

In [670]:
len(factorAPN)

4

In [671]:
boundaryMatrix = getBoundaryMatrix(factorAPN)
coboundaryMatrix = getCoboundaryMatrix(boundaryMatrix)

In [672]:
preAtt = set()
for i in [(0, -1, -2, 1), (0, -2, -1, 1)]:
    apn = factorAPN[i]
    for d in apn:
        preAtt|= apn[d]

In [673]:
tempComplex = set()
for i in factorAPN:
    apn = factorAPN[i]
    for d in apn:
        tempComplex|= apn[d]
tempComplex-=preAtt

subComplex = dict()
for cell in tempComplex:
    subComplex[cell] = set(boundaryMatrix[cell]) - preAtt

In [674]:
queens, kings, aces = genMatchingWithPreAtt(subComplex,coboundaryMatrix,preAtt)

In [675]:

print(len(queens),len(kings),len(aces),len(subComplex))

15 15 0 30


From this and previous boundary connection result "print(s4-s1, s3-s2, s1-s2) = set() set() set()" we can take the polyhedra complex over parameter node (0, -1, -2, 1), (0, -2, -1, 1), which is convex polyhedra and then like Mdm2 we can remove term Wip1 and only consider the polyhedra complex over the two APN

In [520]:
reducedPNList= []
s = set([(0, -1, -2, 1),(0, -2, -1, 1)])
for i in range(len(pnlist)):
    pn = pnlist[i]
    if pn[2] in s:
        reducedPNList.append(pn)
len(reducedPNList)

5016

In [528]:
[reducedPNList[0][1]] + reducedPNList[0][3:]

[(0, -1, -2, 1), (0, 1, 2, -1, 3), (0, 1, 2, 3, 4, 5, 6, -1, -2, 7)]

In [538]:
realReducedPNList = []
for i in range(len(reducedPNList)):
    pn = reducedPNList[i]
    pn = [pn[1]] + pn[3:]
    #print(len(pn))
    realReducedPNList.append(tuple(pn))

In [539]:
realReducedPNList = list(set(realReducedPNList))

In [540]:
len(realReducedPNList)

2508

# Chk2

In [542]:
factorIndex = 3
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = 0

realFactorIndex = 1
for i in range(len(realReducedPNList)):
    currPN = realReducedPNList[i]
    currFactor = currPN[realFactorIndex]
    factorStatistics[currFactor]+=1

In [543]:
factorStatistics

{(0, 1, 2, -1, 3): 627,
 (0, 1, -1, 2, 3): 782,
 (0, 2, -1, 1, 3): 472,
 (0, -1, 1, 2, 3): 627}

In [419]:
factorIndex = 3
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = set()

for i in range(len(newPNList)):
    currPN = newPNList[i]
    currFactor = currPN[factorIndex]
    rest = currPN[:factorIndex] + currPN[factorIndex+1:]
    rest = tuple(rest)
    factorStatistics[currFactor].add(rest)

In [434]:
s1 = factorStatistics[(0, 1, 2, -1, 3)] 
s2 = factorStatistics[(0, 1, -1, 2, 3)]
s3 = factorStatistics[(0, 2, -1, 1, 3)]
s4 = factorStatistics[(0, -1, 1, 2, 3)]

print(s1 - s2, s4 - s2, s3 - s1, s3 - s2)

In [443]:
print(s1 - s2, s4 - s2, s3 - s1, s3 - s2)

set() set() set() set()


In [431]:
# interesting number
len(s4-s1), len(s1-s4)

(310, 310)

In [432]:
apnList = loadAtomicChainComplex('xy_o_1_PN.json')
factorAPN = dict()
for k in factorStatistics:
    factorAPN[k] = apnList[k]

In [435]:
len(factorAPN)

4

In [436]:
boundaryMatrix = getBoundaryMatrix(factorAPN)
coboundaryMatrix = getCoboundaryMatrix(boundaryMatrix)

In [437]:
preAtt = set()
for i in [(0, 1, -1, 2, 3)]:
    apn = factorAPN[i]
    for d in apn:
        preAtt|= apn[d]

In [438]:
tempComplex = set()
for i in factorAPN:
    apn = factorAPN[i]
    for d in apn:
        tempComplex|= apn[d]
tempComplex-=preAtt

subComplex = dict()
for cell in tempComplex:
    subComplex[cell] = set(boundaryMatrix[cell]) - preAtt

In [439]:
queens, kings, aces = genMatchingWithPreAtt(subComplex,coboundaryMatrix,preAtt)

In [440]:
print(len(queens),len(kings),len(aces),len(subComplex))

56 56 0 112


Above computation with the set analysis we can focus on the polyhedra subcomplex that has projection over (0, 1, -1, 2, 3) 

In [447]:
newReducedPNList = [] 
for i in range(len(reducedPNList)):
    pn = reducedPNList[i]
    if pn[3] == (0, 1, -1, 2, 3):
        newReducedPNList.append(pn)
        
len(newReducedPNList)

1564

## ATM

In [450]:
factorIndex = 1
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = 0


for i in range(len(newReducedPNList)):
    currPN = newReducedPNList[i]
    currFactor = currPN[factorIndex]
    factorStatistics[currFactor]+=1

In [451]:
factorStatistics

{(0, -1, -2, 1): 310,
 (0, -2, -1, 1): 310,
 (0, -1, 1, -2): 166,
 (0, -2, 1, -1): 310,
 (-1, 0, -2, 1): 310,
 (-2, 0, -1, 1): 158}

In [455]:
factorIndex = 1
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = set()

for i in range(len(newReducedPNList)):
    currPN = newReducedPNList[i]
    currFactor = currPN[factorIndex]
    rest = currPN[:factorIndex] + currPN[factorIndex+1:]
    rest = tuple(rest)
    factorStatistics[currFactor].add(rest)

In [456]:
s1 = factorStatistics[(0, -1, -2, 1)] 
s2 = factorStatistics[(0, -2, -1, 1)]
s3 = factorStatistics[(0, -1, 1, -2)]
s4 = factorStatistics[(0, -2, 1, -1)]
s5 = factorStatistics[(-1, 0, -2, 1)]
s6 = factorStatistics[(-2, 0, -1, 1)]

In [459]:
print(len(s4-s2), len(s5-s1), len(s3-s1), len(s6-s2))

0 0 18 16


In [461]:
apnList = loadAtomicChainComplex('x_o_2_PN.json')
factorAPN = dict()
for k in factorStatistics:
    factorAPN[k] = apnList[k]
    
len(factorAPN)

6

In [462]:
boundaryMatrix = getBoundaryMatrix(factorAPN)
coboundaryMatrix = getCoboundaryMatrix(boundaryMatrix)

In [463]:
preAtt = set()
# s1,s2, s3, s6
for i in [(0, -1, -2, 1),(0, -2, -1, 1), (0, -1, 1, -2),(-2, 0, -1, 1)]:
    apn = factorAPN[i]
    for d in apn:
        preAtt|= apn[d]

In [464]:
tempComplex = set()
for i in factorAPN:
    apn = factorAPN[i]
    for d in apn:
        tempComplex|= apn[d]
tempComplex-=preAtt

subComplex = dict()
for cell in tempComplex:
    subComplex[cell] = set(boundaryMatrix[cell]) - preAtt

In [465]:
queens, kings, aces = genMatchingWithPreAtt(subComplex,coboundaryMatrix,preAtt)

In [466]:
print(len(queens),len(kings),len(aces),len(subComplex))

15 15 0 30


So from above we can focus on the polyhedra complex that has projection over (0, -1, -2, 1),(0, -2, -1, 1), (0, -1, 1, -2),(-2, 0, -1, 1)

In [477]:
finalPNList = []
for i in range(len(newReducedPNList)):
    pn = newReducedPNList[i]
    if pn[1] in [(0, -1, -2, 1),(0, -2, -1, 1), (0, -1, 1, -2),(-2, 0, -1, 1)]:
        finalPNList.append(pn)

In [480]:
with open('reducePNList.dat','w') as file:
    json.dump(finalPNList,file)

In [481]:
finalPNList = [[x[1],x[-1] ] for x in finalPNList]

# Compute connection matrix over the reduced polyhedra complex

In [ ]:
from homologyHelpers import *
from time import time

   
start = time()
dim = 1
filepathList = ['x_o_2_PN.json','(x+y)z_o_2_PN.json'] 
HChain=dict() 
TheChain=dict()
LChain=dict()
HDiff=dict()
LDiff=dict()
finishedTheChain = set()

nodeTypeFaceLattice = [loadAtomicChainComplex(file) for file in filepathList]

with open('reducePNList.dat','r') as file:
    plist = json.load(file)

generatFLForPNsByDim(finalPNList,nodeTypeFaceLattice, HChain, TheChain,LChain, HDiff,LDiff,dim)

print("The rank chain of dimension {}: {}".format(dim+1,len(HChain)))
print("The rank chain of dimension {}: {}".format(dim,len(TheChain)))
print("The rank chain of dimension {}: {}".format(dim-1,len(LChain)))

print('The nonzeros in sparse matrix from chain {} to chain {}: {}'.format(dim+1, dim, len(HDiff)))
print('The nonzeros in sparse matrix from chain {} to chain {}: {}'.format(dim, dim-1, len(LDiff)))

print("Time for computation: {}".format(time()-start))

#with open('LDiff.dat','w') as file:
#    json.dump(LDiff,file)

#with open('HDiff.dat','w') as file:
#    json.dump(HDiff,file) 

print("Time for all computation and storage: {}".format(time()-start))



In [490]:
len(LChain), len(HChain)

(360, 6234)

In [491]:
with open('reducePNList.dat','r') as file:
    plist = json.load(file)

In [494]:
plist = [[x[1],x[-1]] for x in plist]

# more detailed study of ATM

In [11]:
factorIndex = 1
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = 0

for i in range(len(finalPNList)):
    currPN = finalPNList[i]
    currFactor = currPN[0]
    factorStatistics[currFactor]+=1

NameError: name 'factorList' is not defined

In [507]:
factorStatistics

{(0, -1, -2, 1): 310,
 (0, -2, -1, 1): 310,
 (0, -1, 1, -2): 166,
 (0, -2, 1, -1): 0,
 (-1, 0, -2, 1): 0,
 (-2, 0, -1, 1): 158}

In [515]:
factorIndex = 1
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = set()

for i in range(len(finalPNList)):
    currPN = finalPNList[i]
    currFactor = currPN[0]
    rest = currPN[:0] + currPN[1:]
    rest = tuple(rest)
    factorStatistics[currFactor].add(rest)

In [516]:
s1 = factorStatistics[(0, -1, -2, 1)] 
s2 = factorStatistics[(0, -2, -1, 1)]
s3 = factorStatistics[(0, -1, 1, -2)]
s4 = factorStatistics[(0, -2, 1, -1)]
s5 = factorStatistics[(-1, 0, -2, 1)]
s6 = factorStatistics[(-2, 0, -1, 1)]

In [517]:
print(len(s4-s2), len(s5-s1), len(s3-s1), len(s6-s2))

0 0 9 8


# Real reduced case

In [622]:
len(pnlist)

6904

In [21]:
[len(x) for x in atomicPNList]

[3, 12, 12, 6, 310]

In [19]:
[len(x) for x in factorList]

[1, 6, 4, 4, 172]

In [623]:
# Mdm2
realPNList = [x[1:] for x in pnlist]

In [624]:
# Wip1
factorIndex = 2
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = 0

realFactor = 1
for i in range(len(realPNList)):
    currPN = realPNList[i]
    currFactor = currPN[realFactor]
    factorStatistics[currFactor]+=1

In [625]:
factorStatistics

{(0, -1, -2, 1): 2508,
 (0, -2, -1, 1): 2508,
 (-2, 0, -1, 1): 944,
 (0, -1, 1, -2): 944}

In [626]:
factorIndex = 2
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = set()
    
realFactor = 1
for i in range(len(realPNList)):
    currPN = realPNList[i]
    currFactor = currPN[realFactor]
    rest = currPN[:realFactor] + currPN[realFactor+1:]
    rest = tuple(rest)
    factorStatistics[currFactor].add(rest)

In [627]:
s1 = factorStatistics[(0, -1, -2, 1)]
s2 = factorStatistics[(0, -2, -1, 1)]
s3 = factorStatistics[(-2, 0, -1, 1)]
s4 = factorStatistics[(0, -1, 1, -2)]

In [628]:
print(s4-s1, s3-s2, s1-s2)

set() set() set()


In [675]:
apnList = loadAtomicChainComplex('x_o_2_PN.json')
factorAPN = dict()
for k in factorStatistics:
    factorAPN[k] = apnList[k]
    
s1 = factorStatistics[(0, -1, -2, 1)]
s2 = factorStatistics[(0, -2, -1, 1)]
s3 = factorStatistics[(-2, 0, -1, 1)]
s4 = factorStatistics[(0, -1, 1, -2)]

boundaryMatrix = getBoundaryMatrix(factorAPN)
coboundaryMatrix = getCoboundaryMatrix(boundaryMatrix)

preAtt = set()
for i in [(0, -1, -2, 1), (0, -2, -1, 1)]:
    apn = factorAPN[i]
    for d in apn:
        preAtt|= apn[d]
        
tempComplex = set()
for i in factorAPN:
    apn = factorAPN[i]
    for d in apn:
        tempComplex|= apn[d]
tempComplex-=preAtt

subComplex = dict()
for cell in tempComplex:
    subComplex[cell] = set(boundaryMatrix[cell]) - preAtt
    
queens, kings, aces = genMatchingWithPreAtt(subComplex,coboundaryMatrix,preAtt)


print(len(queens),len(kings),len(aces),len(subComplex))

15 15 0 30


In [573]:
realNewPNList = []
for i in range(len(realPNList)):
    pn = realPNList[i]
    if pn[1] in [(0, -1, -2, 1), (0, -2, -1, 1)]:
        realNewPNList.append(tuple([pn[0]]+pn[2:]))

In [575]:
realNewPNList = list(set(realNewPNList))

In [576]:
len(realNewPNList)

2508

In [577]:
# Chk2
factorIndex = 3
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = 0

realFactor = 1
for i in range(len(realNewPNList)):
    currPN = realNewPNList[i]
    currFactor = currPN[realFactor]
    factorStatistics[currFactor]+=1

In [578]:
factorStatistics

{(0, 1, 2, -1, 3): 627,
 (0, 1, -1, 2, 3): 782,
 (0, 2, -1, 1, 3): 472,
 (0, -1, 1, 2, 3): 627}

In [579]:
factorIndex = 3
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = set()

realFactor = 1
for i in range(len(realNewPNList)):
    currPN = realNewPNList[i]
    currFactor = currPN[realFactor]
    rest = currPN[:realFactor] + currPN[realFactor+1:]
    rest = tuple(rest)
    factorStatistics[currFactor].add(rest)

In [583]:
s1 = factorStatistics[(0, 1, 2, -1, 3)] 
s2 = factorStatistics[(0, 1, -1, 2, 3)]
s3 = factorStatistics[(0, 2, -1, 1, 3)]
s4 = factorStatistics[(0, -1, 1, 2, 3)]

print(s1 - s2, s4 - s2, s3 - s1, s3 - s4, s3 - s2)

set() set() set() set() set()


In [440]:
apnList = loadAtomicChainComplex('xy_o_1_PN.json')
factorAPN = dict()
for k in factorStatistics:
    factorAPN[k] = apnList[k]
    
boundaryMatrix = getBoundaryMatrix(factorAPN)
coboundaryMatrix = getCoboundaryMatrix(boundaryMatrix)

preAtt = set()
for i in [(0, 1, -1, 2, 3)]:
    apn = factorAPN[i]
    for d in apn:
        preAtt|= apn[d]
    
tempComplex = set()
for i in factorAPN:
    apn = factorAPN[i]
    for d in apn:
        tempComplex|= apn[d]
tempComplex-=preAtt

subComplex = dict()
for cell in tempComplex:
    subComplex[cell] = set(boundaryMatrix[cell]) - preAtt

queens, kings, aces = genMatchingWithPreAtt(subComplex,coboundaryMatrix,preAtt)


print(len(queens),len(kings),len(aces),len(subComplex))

56 56 0 112


In [593]:
realNewNewPNList = []
for i in range(len(realNewPNList)):
    pn = realNewPNList[i]
    if pn[1] in [(0, 1, -1, 2, 3)]:
        realNewNewPNList.append(tuple([pn[0]]+list(pn[2:])))

In [595]:
realNewNewPNList = list(set(realNewNewPNList))

In [598]:
# ATM
factorIndex = 1
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = 0

realFactor = 0
for i in range(len(realNewNewPNList)):
    currPN = realNewNewPNList[i]
    currFactor = currPN[realFactor]
    factorStatistics[currFactor]+=1

In [599]:
factorStatistics

{(0, -1, -2, 1): 155,
 (0, -2, -1, 1): 155,
 (0, -1, 1, -2): 83,
 (0, -2, 1, -1): 155,
 (-1, 0, -2, 1): 155,
 (-2, 0, -1, 1): 79}

In [603]:
factorIndex = 1
s = factorList[factorIndex]
factorStatistics = dict()
for e in s:
    factorStatistics[e] = set()

realFactor = 0
for i in range(len(realNewNewPNList)):
    currPN = realNewNewPNList[i]
    currFactor = currPN[realFactor]
    rest = currPN[:realFactor] + currPN[realFactor+1:]
    rest = tuple(rest)
    factorStatistics[currFactor].add(rest)

In [604]:
s1 = factorStatistics[(0, -1, -2, 1)] 
s2 = factorStatistics[(0, -2, -1, 1)]
s3 = factorStatistics[(0, -1, 1, -2)]
s4 = factorStatistics[(0, -2, 1, -1)]
s5 = factorStatistics[(-1, 0, -2, 1)]
s6 = factorStatistics[(-2, 0, -1, 1)]

In [605]:
print(len(s4-s2), len(s5-s1), len(s3-s1), len(s6-s2))

0 0 9 8


In [466]:
apnList = loadAtomicChainComplex('x_o_2_PN.json')
factorAPN = dict()
for k in factorStatistics:
    factorAPN[k] = apnList[k]

boundaryMatrix = getBoundaryMatrix(factorAPN)
coboundaryMatrix = getCoboundaryMatrix(boundaryMatrix)

preAtt = set()
# s1,s2, s3, s6
for i in [(0, -1, -2, 1),(0, -2, -1, 1), (0, -1, 1, -2),(-2, 0, -1, 1)]:
    apn = factorAPN[i]
    for d in apn:
        preAtt|= apn[d]

tempComplex = set()
for i in factorAPN:
    apn = factorAPN[i]
    for d in apn:
        tempComplex|= apn[d]
tempComplex-=preAtt

subComplex = dict()
for cell in tempComplex:
    subComplex[cell] = set(boundaryMatrix[cell]) - preAtt

queens, kings, aces = genMatchingWithPreAtt(subComplex,coboundaryMatrix,preAtt)

print(len(queens),len(kings),len(aces),len(subComplex))

15 15 0 30


In [606]:
hasCommonFace = (s3&s1)
noCommonFace = s3 - s1

In [608]:
len(hasCommonFace), len(noCommonFace)

(74, 9)

In [611]:
realFinalPNList = []
for i in range(len(realNewNewPNList)):
    pn = realNewNewPNList[i]
    if pn[0] in [(0, -1, -2, 1),(0, -2, -1, 1), (0, -1, 1, -2),(-2, 0, -1, 1)]:
        realFinalPNList.append(pn)

In [612]:
len(realFinalPNList)

472

In [613]:
310+83+79

472

In [615]:
with open('essentialPNList.dat','w') as file:
    json.dump(realFinalPNList,file)

In [616]:
realFinalPNList

[((0, -1, -2, 1), (0, 1, 2, 3, 4, -1, 5, -2, 6, 7)),
 ((0, -2, -1, 1), (0, 1, 2, 3, 4, -1, 5, -2, 6, 7)),
 ((0, -1, -2, 1), (0, 1, 2, 3, 4, -2, 5, -1, 6, 7)),
 ((0, -2, -1, 1), (0, 1, 2, 3, 4, -2, 5, -1, 6, 7)),
 ((0, -1, -2, 1), (0, 1, 2, -2, 3, 4, 6, -1, 5, 7)),
 ((0, -2, -1, 1), (0, 1, 2, -2, 3, 4, 6, -1, 5, 7)),
 ((0, -1, -2, 1), (0, 4, -2, 2, 6, -1, 1, 3, 5, 7)),
 ((0, -2, -1, 1), (0, 4, -2, 2, 6, -1, 1, 3, 5, 7)),
 ((0, -1, -2, 1), (0, 1, -1, 2, 4, 5, -2, 3, 6, 7)),
 ((0, -2, -1, 1), (0, 1, 2, 4, 6, -1, 5, -2, 3, 7)),
 ((0, -1, -2, 1), (0, 1, 2, 4, 6, -2, 5, -1, 3, 7)),
 ((0, -2, -1, 1), (0, 1, 2, 4, 6, -2, 5, -1, 3, 7)),
 ((0, -1, -2, 1), (0, 1, 2, 4, 6, -1, 5, -2, 3, 7)),
 ((0, -2, -1, 1), (0, 1, -1, 2, 4, 5, -2, 3, 6, 7)),
 ((0, -1, -2, 1), (0, 1, -2, 2, 4, 5, -1, 3, 6, 7)),
 ((0, -2, -1, 1), (0, 1, -2, 2, 4, 5, -1, 3, 6, 7)),
 ((0, -1, 1, -2), (0, 1, 2, 4, 6, -1, 3, -2, 5, 7)),
 ((0, -1, 1, -2), (0, 1, 2, 4, 6, -2, 3, -1, 5, 7)),
 ((-2, 0, -1, 1), (0, 2, 4, 6, -1, -2, 1, 3, 5

In [3]:
import json
temp= loadAtomicChainComplex('(x+y)z_o_2_PN.json')

In [5]:
list(temp.keys())[0]

(0, 1, 4, 5, -2, 2, 6, -1, 3, 7)

In [8]:
temp[(0, 1, 4, 5, -2, 2, 6, -1, 3, 7)]

{0: {(73,), (77,), (206,), (217,), (250,), (292,), (327,), (442,)},
 1: {(73, 77),
  (73, 206),
  (73, 217),
  (73, 250),
  (73, 292),
  (73, 327),
  (73, 442),
  (77, 206),
  (77, 217),
  (77, 250),
  (77, 292),
  (77, 327),
  (77, 442),
  (206, 217),
  (206, 250),
  (206, 292),
  (206, 327),
  (206, 442),
  (217, 250),
  (217, 292),
  (217, 327),
  (217, 442),
  (250, 292),
  (250, 327),
  (250, 442),
  (292, 327),
  (292, 442),
  (327, 442)},
 2: {(73, 77, 206),
  (73, 77, 217),
  (73, 77, 250),
  (73, 77, 292),
  (73, 77, 327),
  (73, 77, 442),
  (73, 206, 217),
  (73, 206, 250),
  (73, 206, 292),
  (73, 206, 327),
  (73, 206, 442),
  (73, 217, 250),
  (73, 217, 292),
  (73, 217, 327),
  (73, 217, 442),
  (73, 250, 292),
  (73, 250, 327),
  (73, 250, 442),
  (73, 292, 327),
  (73, 292, 442),
  (73, 327, 442),
  (77, 206, 217),
  (77, 206, 250),
  (77, 206, 292),
  (77, 206, 327),
  (77, 206, 442),
  (77, 217, 250),
  (77, 217, 292),
  (77, 217, 327),
  (77, 217, 442),
  (77, 250, 2

In [9]:
1+1

2

In [10]:
pnlist

NameError: name 'pnlist' is not defined

# more detail of p53 parameter node

In [17]:
pnlist = []
with open('essentialPNList.dat','r') as file:
    pnlist = json.load(file)

In [20]:
for i in range(len(pnlist)):
    pnlist[i] = [tuple(x) for x in pnlist[i]]

In [22]:
factorStatistics = dict()
for i in range(len(pnlist)):
    pn = pnlist[i]
    if pn[1] not in factorStatistics:
        factorStatistics[pn[1]] = set()
    factorStatistics[pn[1]].add(pn[0])

In [23]:
len(factorStatistics)

172

In [24]:
def checkDifferenceDegree(apnIndex1, apnIndex2):
    degree = 0
    thresholds = sorted([i for i in apnIndex1 if i <0])
    
    if len(thresholds) == 2: # check for two threhold cases
        first = (apnIndex1.index(thresholds[0]) < apnIndex1.index(thresholds[1]))
        second = (apnIndex1.index(thresholds[0]) < apnIndex1.index(thresholds[1]))

        if first != second:
            degree+=1
    
    polys = [i for i in range(len(apnIndex1) - len(thresholds))]
    if degree == 0:
        for poly in polys:
            firstLogic = [apnIndex1.index(poly) < apnIndex1.index(t) for t in thresholds]
            secondLogic = [apnIndex2.index(poly) < apnIndex1.index(t) for t in thresholds]
            if firstLogic != secondLogic:
                degree+=1
    else: # reverse the thresholds when comparing this is only for 1,2 threholds
        for poly in polys:
            firstLogic = [apnIndex1.index(poly) < apnIndex1.index(t) for t in thresholds]
            secondLogic = [apnIndex2.index(poly) < apnIndex1.index(t) for t in thresholds[::-1]]
            if firstLogic != secondLogic:
                degree+=1
            
    return degree
            
        
def checkGraphicalAdj(pnIndex1, pnIndex2):
    deg = 0
    for i in range(len(pnIndex1)):
        deg += checkDifferenceDegree(pnIndex1[i],pnIndex2[i])
        if deg >1:
            return False
    return True

In [25]:
checkGraphicalAdj(pnlist[0],pnlist[1])

True

In [41]:
projectedPNList = set()
for i in pnlist:
    projectedPNList.add((i[1],))
projectedPNList = list(projectedPNList)
len(projectedPNList)

172

In [43]:
projectedPNList[0]

((0, 1, 2, 3, -1, -2, 4, 5, 6, 7),)

In [44]:
graph = dict()
for i in range(len(projectedPNList)):
    for j in range(i+1,len(projectedPNList)):
        if checkGraphicalAdj(projectedPNList[i],projectedPNList[j]):
            if i not in graph:
                graph[i] = set()
            graph[i].add(j)
             
            if j not in graph:
                graph[j] = set()
            graph[j].add(i)

In [45]:
len(graph)

172

In [51]:
degree = []
for i in range(len(graph)):
    degree.append(len(graph[i]))

In [57]:
degree.index(0)

ValueError: 0 is not in list

In [70]:
cylinderDict = dict()
for i in range(len(projectedPNList)):
    projectedPN = projectedPNList[i][0]
    for j in range(len(pnlist)):
        
        if projectedPN not in cylinderDict:
            cylinderDict[projectedPN] = set()
            
        if pnlist[j][1] == projectedPN:
            cylinderDict[projectedPN].add(pnlist[j][0])
        

In [73]:
sum([len(cylinderDict[x]) for x in cylinderDict])

472

In [78]:
def isCombinatoricallyClose(projectedPNIndex,projectedPNList,graph,cylinderDict):
    projectedAdjacent = graph[projectedPNIndex]
    projectedPN = projectedPNList[projectedPNIndex][0]
    
    for i in projectedAdjacent:
        nghd = projectedPNList[i][0]
        if not cylinderDict[projectedPN].issubset(cylinderDict[nghd]):
            return False
    
    return True

In [79]:
degree.index(10)

44

In [80]:
isCombinatoricallyClose(44,projectedPNList,graph,cylinderDict)

True

In [82]:
candidatesCClosed = []
for i in range(len(projectedPNList)):
    candidatesCClosed.append(isCombinatoricallyClose(i,projectedPNList,graph,cylinderDict))

In [84]:
candidatesCClosed.count(True)

26

In [85]:
candidateIndexList = []
for i in range(len(candidatesCClosed)):
    if candidatesCClosed[i]:
        candidateIndexList.append(i)

In [87]:
candidateProjectedPNList = [projectedPNList[i][0] for i in candidateIndexList]

In [91]:
count = 0
for i in range(len(candidateProjectedPNList)):
    projectedPN = candidateProjectedPNList[i]
    count += len(cylinderDict[projectedPN])

In [92]:
count

52

In [9]:
s1

NameError: name 's1' is not defined